# Linear Relaxation Algorithm

In [ ]:
import numpy as np
from scipy.optimize import linprog

def linear_relaxation(c, A, b):
    # Solve the linear programming problem
    res = linprog(c, A_ub=A, b_ub=b, bounds=(0, None))
    return res.x

# Example usage
c = np.array([2, 3])  # Objective coefficients
A = np.array([[1, 1], [1, -1]])  # Constraint matrix
b = np.array([4, 1])  # Constraint vector

optimal_solution = linear_relaxation(c, A, b)
print("Optimal solution:", optimal_solution)

# Lagrangian Relaxation Algorithm

In [ ]:
import numpy as np
from scipy.optimize import minimize

def lagrangian_dual(c, A, b):
    # Define Lagrangian function
    def lagrangian(x, l):
        return np.dot(c, x) + np.dot(l, np.dot(A, x) - b)

    # Initial guess for Lagrange multipliers
    initial_lambda = np.zeros(len(b))

    # Define Lagrange multiplier update function
    def update_lambda(l):
        res = minimize(lambda l: -lagrangian(np.zeros_like(c), l), l, bounds=[(0, None) for _ in range(len(b))])
        return res.x

    # Initialize Lagrange multipliers
    lambdas = initial_lambda

    # Iteratively update Lagrange multipliers until convergence
    while True:
        new_lambdas = update_lambda(lambdas)
        if np.allclose(new_lambdas, lambdas):
            break
        lambdas = new_lambdas

    return lambdas

# Example usage
c = np.array([2, 3])  # Objective coefficients
A = np.array([[1, 1], [1, -1]])  # Constraint matrix
b = np.array([4, 1])  # Constraint vector

optimal_lambdas = lagrangian_dual(c, A, b)
print("Optimal Lagrange multipliers:", optimal_lambdas)

# Cutting Plane Method

In [ ]:
import numpy as np
from scipy.optimize import linprog

class LPProblem:
    def __init__(self, c, A_ub, b_ub):
        self.c = c  # Objective coefficients
        self.A_ub = A_ub  # Constraint matrix for upper bounds
        self.b_ub = b_ub  # Right-hand side vector for upper bounds
        self.cut_planes = []  # List to store cutting planes

    def has_fractional_solution(self, x):
        # Check if any variable in the solution is fractional
        return any(abs(xi - np.round(xi)) > 1e-6 for xi in x)

    def solve(self):
        # Solve the LP relaxation
        res = linprog(self.c, A_ub=self.A_ub, b_ub=self.b_ub, bounds=(0, None))
        return res.x

    def add_cutting_plane(self, cutting_plane):
        # Add the cutting plane to the LP relaxation
        self.cut_planes.append(cutting_plane)
        self.A_ub = np.vstack([self.A_ub, cutting_plane])
        self.b_ub = np.append(self.b_ub, cutting_plane[-1])

    def optimal_solution(self):
        # Solve the updated LP relaxation
        res = linprog(self.c, A_ub=self.A_ub, b_ub=self.b_ub, bounds=(0, None))
        return res.x

def derive_cutting_plane(fractional_solution):
    # Derive a cutting plane from the fractional solution
    # For demonstration purposes, assume a simple example
    cutting_plane = np.array([1, 1])  # Adjust as per the problem requirements
    return cutting_plane

def cutting_plane_method(LP_relaxation):
    while LP_relaxation.has_fractional_solution(LP_relaxation.solve()):
        fractional_solution = LP_relaxation.solve()
        cutting_plane = derive_cutting_plane(fractional_solution)
        LP_relaxation.add_cutting_plane(cutting_plane)
    return LP_relaxation.optimal_solution()

# Example usage
c = np.array([2, 3])  # Objective coefficients
A_ub = np.array([[1, 1], [1, -1]])  # Constraint matrix for upper bounds
b_ub = np.array([4, 1])  # Right-hand side vector for upper bounds

lp_problem = LPProblem(c, A_ub, b_ub)
optimal_solution = cutting_plane_method(lp_problem)
print("Optimal solution:", optimal_solution)

# Subgradient Optimization Algorithm

In [ ]:
import numpy as np

def subgradient_method(f, grad_f, x0, step_size=0.1, max_iter=1000, tol=1e-6):
    """
    Subgradient Optimization Algorithm for minimizing a convex function f(x).

    Parameters:
    - f: Function to minimize.
    - grad_f: Function to compute the subgradient of f(x) at x.
    - x0: Initial point.
    - step_size: Step size or learning rate.
    - max_iter: Maximum number of iterations.
    - tol: Tolerance for convergence (stops if ||x_new - x_old|| < tol).

    Returns:
    - x_opt: Optimal point that minimizes f(x).
    - f_opt: Minimum value of f(x) at x_opt.
    - iter_count: Number of iterations taken to converge.
    """
    x = x0
    iter_count = 0
    while iter_count < max_iter:
        # Compute subgradient of f(x) at current point x
        g = grad_f(x)

        # Update x using subgradient descent
        x_new = x - step_size * g

        # Check convergence criterion
        if np.linalg.norm(x_new - x) < tol:
            break

        # Update x and iteration count
        x = x_new
        iter_count += 1

    # Compute the optimal value of f(x) at x_opt
    x_opt = x
    f_opt = f(x_opt)

    return x_opt, f_opt, iter_count

# Example usage
def f(x):
    return np.sum(np.square(x))  # Example convex function: sum of squares

def grad_f(x):
    return 2 * x  # Gradient of sum of squares

# Initial point and parameters
x0 = np.array([1.0, 2.0])
step_size = 0.1
max_iter = 1000
tol = 1e-6

# Apply Subgradient Optimization Algorithm
x_opt, f_opt, iter_count = subgradient_method(f, grad_f, x0, step_size, max_iter, tol)

print("Optimal point x:", x_opt)
print("Minimum value f(x):", f_opt)
print("Iterations:", iter_count)

# SDP Relaxation Algorithm

In [ ]:
import numpy as np
import cvxpy as cp

# Example problem: Max-Cut problem relaxation using SDP
def sdp_relaxation(W):
    n = W.shape[0]  # Number of nodes

    # Variables
    X = cp.Variable((n, n), symmetric=True)

    # Objective
    objective = cp.Maximize(cp.sum(cp.multiply(W, X)))

    # Constraints
    constraints = [
        X >> 0,  # X is positive semidefinite
        cp.diag(X) == np.ones(n),  # Diagonal entries of X are 1
    ]

    # Define and solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.SCS)  # Use SCS solver for SDP

    # Retrieve the optimal solution
    X_opt = X.value

    return X_opt

# Example usage
if __name__ == '__main__':
    # Generate an example adjacency matrix for a graph
    W = np.array([
        [0, 1, 1, 0],
        [1, 0, 1, 1],
        [1, 1, 0, 1],
        [0, 1, 1, 0]
    ])

    # Solve SDP relaxation
    X_opt = sdp_relaxation(W)

    # Print the optimal solution (approximated solution)
    print("Optimal SDP solution:")
    print(X_opt)

# Ford-Fulkerson Algorithm

In [ ]:
from collections import deque

def ford_fulkerson(graph, s, t):
    def bfs(s, t, parent):
        visited = [False] * len(graph)
        queue = deque()
        queue.append(s)
        visited[s] = True

        while queue:
            u = queue.popleft()
            for v, residual in enumerate(graph[u]):
                if not visited[v] and residual > 0:
                    queue.append(v)
                    visited[v] = True
                    parent[v] = u
                    if v == t:
                        return True
        return False

    parent = [-1] * len(graph)
    max_flow = 0

    while bfs(s, t, parent):
        path_flow = float('Inf')
        v = t
        while v != s:
            u = parent[v]
            path_flow = min(path_flow, graph[u][v])
            v = parent[v]

        # update residual capacities of the edges and reverse edges
        v = t
        while v != s:
            u = parent[v]
            graph[u][v] -= path_flow
            graph[v][u] += path_flow
            v = parent[v]

        max_flow += path_flow

    return max_flow

# Example usage:
# Define the graph as an adjacency matrix
graph = [
    [0, 16, 13, 0, 0, 0],
    [0, 0, 10, 12, 0, 0],
    [0, 4, 0, 0, 14, 0],
    [0, 0, 9, 0, 0, 20],
    [0, 0, 0, 7, 0, 4],
    [0, 0, 0, 0, 0, 0]
]

source = 0  # source node
sink = 5    # sink node

max_flow = ford_fulkerson(graph, source, sink)
print("Maximum flow from source to sink:", max_flow)

# Earliest Due Date (EDD) Algorithm

In [ ]:
def earliest_due_date(tasks):
    # Sort tasks by their due dates (ascending order)
    sorted_tasks = sorted(tasks, key=lambda x: x['due_date'])

    # Print the schedule based on EDD
    print("Task Schedule based on Earliest Due Date (EDD):")
    for task in sorted_tasks:
        print(f"Task {task['id']} - Due Date: {task['due_date']}")

# Example usage:
if __name__ == "__main__":
    # Define tasks with their IDs and due dates
    tasks = [
        {'id': 1, 'due_date': 10},
        {'id': 2, 'due_date': 5},
        {'id': 3, 'due_date': 8},
        {'id': 4, 'due_date': 15},
        {'id': 5, 'due_date': 3}
    ]

    # Call the EDD function to schedule tasks
    earliest_due_date(tasks)

# Nearest Neighbor Heuristic

In [ ]:
import numpy as np

def nearest_neighbor(matrix):
    n = len(matrix)
    unvisited = set(range(1, n))  # set of unvisited nodes, excluding the start node (0)
    current_node = 0              # start with node 0
    tour = [current_node]         # initial tour starts with node 0
    total_cost = 0

    while unvisited:
        nearest_node = min(unvisited, key=lambda x: matrix[current_node][x])
        total_cost += matrix[current_node][nearest_node]
        current_node = nearest_node
        tour.append(current_node)
        unvisited.remove(current_node)

    # Complete the tour by returning to the starting node
    tour.append(0)
    total_cost += matrix[current_node][0]

    return tour, total_cost

# Example usage:
if __name__ == "__main__":
    # Example adjacency matrix for a TSP problem
    adjacency_matrix = np.array([
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ])

    # Apply nearest neighbor heuristic
    tour, cost = nearest_neighbor(adjacency_matrix)

    # Output results
    print("Nearest Neighbor Heuristic Tour:", tour)
    print("Total Cost:", cost)

# Branch and Bound Algorithm for TSP

In [ ]:
import numpy as np
import heapq

class Node:
    def __init__(self, path, cost, bound):
        self.path = path        # List of cities visited so far
        self.cost = cost        # Cost of the path
        self.bound = bound      # Lower bound of the node's path cost

def tsp_branch_and_bound(matrix):
    n = len(matrix)

    # Priority queue (min-heap) to store nodes
    priority_queue = []

    # Initialize the root node
    root = Node([0], 0, calculate_bound([0], matrix))
    best_cost = float('inf')
    best_path = None

    heapq.heappush(priority_queue, (root.bound, root))

    while priority_queue:
        # Extract node with the minimum bound
        _, node = heapq.heappop(priority_queue)

        # If the node represents a complete tour
        if len(node.path) == n:
            # Add the cost to return to the starting city
            total_cost = node.cost + matrix[node.path[-1]][0]
            # Update the best solution if found
            if total_cost < best_cost:
                best_cost = total_cost
                best_path = node.path + [0]
        else:
            # Branch on the node
            for city in range(n):
                if city not in node.path:
                    new_path = node.path + [city]
                    new_cost = node.cost + matrix[node.path[-1]][city]
                    new_bound = calculate_bound(new_path, matrix)

                    # Add child node to the priority queue if promising
                    if new_bound < best_cost:
                        new_node = Node(new_path, new_cost, new_bound)
                        heapq.heappush(priority_queue, (new_node.bound, new_node))

    return best_path, best_cost

def calculate_bound(path, matrix):
    n = len(matrix)
    bound = 0

    # Calculate the cost of the current path
    for i in range(len(path) - 1):
        bound += matrix[path[i]][path[i+1]]

    # Add the lower bound for remaining nodes
    last_city = path[-1]
    bound += min(matrix[last_city][city] for city in range(n) if city not in path)

    return bound

# Example usage:
if __name__ == "__main__":
    # Example adjacency matrix for a TSP problem
    adjacency_matrix = np.array([
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ])

    # Apply Branch and Bound algorithm
    path, cost = tsp_branch_and_bound(adjacency_matrix)

    # Output results
    print("Optimal Tour:", path)
    print("Total Cost:", cost)

# Quadratic Programming Relaxation

In [ ]:
import numpy as np
import cvxopt

def qp_relaxation(Q, c, A=None, b=None, lb=None, ub=None):
    # Convert numpy arrays to cvxopt matrices
    Q = cvxopt.matrix(Q)
    c = cvxopt.matrix(c)

    if A is not None:
        A = cvxopt.matrix(A)
    if b is not None:
        b = cvxopt.matrix(b)
    if lb is not None:
        lb = cvxopt.matrix(lb)
    if ub is not None:
        ub = cvxopt.matrix(ub)

    # Setup the QP problem
    sol = cvxopt.solvers.qp(Q, c, A, b, lb, ub)

    # Extract the solution
    x_opt = np.array(sol['x']).flatten()
    return x_opt

# Example usage:
if __name__ == "__main__":
    # Example QP problem:
    # Minimize: 0.5 * x^T * Q * x + c^T * x
    # Subject to: A * x <= b, lb <= x <= ub

    # Define the QP parameters
    Q = np.array([[2, 0], [0, 2]])  # Quadratic coefficient matrix
    c = np.array([0, 0])            # Linear coefficient vector
    A = np.array([[-1, 0], [0, -1]])  # Constraint matrix
    b = np.array([0, 0])            # Constraint vector
    lb = np.array([-10, -10])       # Lower bounds
    ub = np.array([10, 10])         # Upper bounds

    # Solve the QP relaxation
    x_opt = qp_relaxation(Q, c, A, b, lb, ub)

    # Print the optimal solution
    print("Optimal Solution:")
    print(x_opt)

# Nearest Neighbor Algorithm for TSP

In [ ]:
import numpy as np

def nearest_neighbor_tsp(distances):
    num_cities = len(distances)
    visited = [False] * num_cities
    tour = []

    # Start from city 0 (can start from any city)
    current_city = 0
    tour.append(current_city)
    visited[current_city] = True

    # Iteratively visit the nearest unvisited city
    for _ in range(num_cities - 1):
        nearest_city = None
        min_distance = float('inf')

        # Find nearest unvisited city
        for next_city in range(num_cities):
            if not visited[next_city] and distances[current_city][next_city] < min_distance:
                nearest_city = next_city
                min_distance = distances[current_city][next_city]

        # Move to the nearest city
        current_city = nearest_city
        tour.append(current_city)
        visited[current_city] = True

    # Return to the starting city to complete the tour
    tour.append(tour[0])

    return tour

# Example usage:
if __name__ == "__main__":
    # Example distance matrix (symmetric)
    distances = np.array([
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ])

    # Solve TSP using Nearest Neighbor Algorithm
    tour = nearest_neighbor_tsp(distances)

    # Print the tour
    print("Optimal Tour:", tour)

# Parallel Jacobi Iteration

In [ ]:
import numpy as np
import multiprocessing

def parallel_jacobi(A, b, x0, epsilon):
    n = len(A)
    x = x0.copy()
    while True:
        x_new = np.zeros_like(x)

        # Define a function for parallel computation
        def update_x(i):
            return (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

        # Use multiprocessing for parallel computation
        with multiprocessing.Pool() as pool:
            x_new = np.array(pool.map(update_x, range(n)))

        # Check convergence
        if np.linalg.norm(x_new - x) < epsilon:
            break

        x = x_new

    return x

# Example usage:
if __name__ == "__main__":
    A = np.array([[10, -1, 2], [-1, 11, -1], [2, -1, 10]])
    b = np.array([6, 25, -11])
    x0 = np.zeros_like(b)
    epsilon = 1e-6

    solution = parallel_jacobi(A, b, x0, epsilon)
    print("Solution:", solution)

# Relaxation-based Heuristic for CVRP

In [ ]:
import numpy as np
import copy

def relaxation_heuristic_customers(customers, num_vehicles, vehicle_capacity):
    routes = [[] for _ in range(num_vehicles)]
    for customer in customers:
        min_route = min(routes, key=lambda route: route_demand(route) + customer.demand)
        if route_demand(min_route) + customer.demand <= vehicle_capacity:
            min_route.append(customer)
        else:
            routes.append([customer])
    return routes

def route_demand(route):
    return sum([customer.demand for customer in route])

# Example usage:
if __name__ == "__main__":
    # Define example data
    customers = [
        {'demand': 1}, {'demand': 2}, {'demand': 1}, {'demand': 3}, {'demand': 2}, {'demand': 4},
        {'demand': 2}, {'demand': 3}, {'demand': 1}, {'demand': 2}, {'demand': 1}, {'demand': 3}
    ]
    num_vehicles = 3
    vehicle_capacity = 5

    routes = relaxation_heuristic_customers(customers, num_vehicles, vehicle_capacity)

    # Print the routes
    for i, route in enumerate(routes):
        print(f"Route {i+1}: {[customer['demand'] for customer in route]}")

# Relaxation-based Power Control Algorithm

In [ ]:
import numpy as np

def relaxation_based_power_control(num_devices, max_power, tolerance=1e-6):
    # Initialize transmission powers randomly
    transmission_powers = np.random.uniform(0, max_power, num_devices)

    while True:
        # Relax constraints (none initially)

        # Solve relaxed problem to obtain optimal powers (randomly changing powers for demonstration)
        new_powers = np.random.uniform(0, max_power, num_devices)

        # Tighten constraints (none initially)

        # Check convergence
        if np.linalg.norm(new_powers - transmission_powers) < tolerance:
            break

        # Update transmission powers
        transmission_powers = new_powers

    return transmission_powers

# Example usage:
if __name__ == "__main__":
    num_devices = 5
    max_power = 10.0

    optimal_powers = relaxation_based_power_control(num_devices, max_power)
    print("Optimal transmission powers:", optimal_powers)

# Relaxation-based Mean-Variance Portfolio Optimization

In [ ]:
import numpy as np

def relaxation_based_portfolio_optimization(expected_returns, cov_matrix, tolerance=1e-6):
    num_assets = len(expected_returns)
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)  # Normalize initial weights to sum to 1

    while True:
        # Relax constraints (none initially)

        # Solve relaxed problem to obtain optimal weights
        new_weights = np.random.random(num_assets)
        new_weights /= np.sum(new_weights)  # Normalize new weights

        # Tighten constraints (none initially)

        # Check convergence
        if np.linalg.norm(new_weights - weights) < tolerance:
            break

        # Update weights
        weights = new_weights

    return weights

# Example usage:
if __name__ == "__main__":
    # Example data: expected returns and covariance matrix
    expected_returns = np.array([0.05, 0.07, 0.06])
    cov_matrix = np.array([[0.04, 0.006, -0.01],
                           [0.006, 0.05, 0.004],
                           [-0.01, 0.004, 0.062]])

    optimal_weights = relaxation_based_portfolio_optimization(expected_returns, cov_matrix)
    print("Optimal portfolio weights:", optimal_weights)